In [1]:
import aocd
import numpy as np
from anytree import Node
import json
import math
from functools import reduce
from itertools import permutations
from copy import deepcopy
%run helper.ipynb
puzzle = aocd.models.Puzzle(year=2021, day=18)
data = puzzle.input_data.split("\n")

In [2]:
def string_to_tree(string):
    j = json.loads(string)
    if(isinstance(j[0], int)):
        left = Node(j[0])
    else:
        left = string_to_tree(str(j[0]))
    if(isinstance(j[1], int)):
        right = Node(j[1])
    else:
        right = string_to_tree(str(j[1]))
    ret = Node("")
    ret.children = [left, right]
    return ret

In [3]:
def get_side(n, idx):
    last_node = n
    for node in n.parent.iter_path_reverse():
        if(node.children[1-idx] == last_node):
            return node.children[idx].leaves[idx-1]
        else:
            last_node = node
    else:
        return None

In [4]:
def reduce_tree(tree):
    e = [x for x in tree.leaves if x.depth > 4]
    if(len(e) > 0):
        p = e[0].parent
        (l,r) = p.children
        ll = get_side(l, 0)
        if(ll is not None):
            ll.name = int(ll.name) + int(l.name)
        rr = get_side(r, 1)
        if(rr is not None):
            rr.name = int(rr.name) + int(r.name)
        p.name = 0
        p.children = []
        return reduce_tree(tree)
    g = [x for x in tree.leaves if int(x.name) >= 10]
    if(len(g) > 0):
        s = g[0]
        v = int(s.name)
        s.name = ""
        s.children = [Node(math.floor(v/2)), Node(math.ceil(v/2))]
        return reduce_tree(tree)
    return tree
        

In [5]:
def add(n1, n2):
    return reduce_tree(Node("", children=[n1, n2]))

In [6]:
def magnitude(tree):
    if tree.is_leaf:
        return int(tree.name)
    else:
        return 3 * magnitude(tree.children[0]) + 2 * magnitude(tree.children[1])

In [7]:
puzzle.answer_a = magnitude(reduce(add, map(string_to_tree, data)))

In [8]:
puzzle.answer_b = max([magnitude(add(string_to_tree(t[0]), string_to_tree(t[1]))) for t in permutations(data, 2)])